# Mission To Mars

#### Purpose: build a web application that scrapes various websites for data related to the Mission to Mars and displays the information in a single HTML page

## Step 1: Web Scrape

In [10]:
# Import dependencies 
import os
from bs4 import BeautifulSoup as bs
import urllib
import requests
from splinter import Browser


In [34]:
mars_page_url = 'https://mars.nasa.gov/news/'
mars_page = requests.get(mars_page_url)
soup = bs(mars_page.text, 'lxml')

results = soup.find_all('ul', class_="item_list")

for result in results:
    # Error handling
    try:
        # Identify and return title of listing
        teaser_title = result.find('div', class_="content_title").text
        print(teaser_title)
        # Identify and return price of listing
        teaser_body = result.find('div', class_="article_teaser_body").text
        print(teaser_body)
    except AttributeError as e:
        print(e)



In [ ]:
filepath = os.path.join("..", "Resources", "template.html")
with open(filepath) as file:
    html = file.read()

In [ ]:
browser = Browser('chrome')
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
for x in range(1):

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    quotes = soup.find_all('span', class_='text')

    for quote in quotes:
        print('page:', x, '-------------')
        print(quote.text)

    browser.click_link_by_partial_text('Next')